<a href="https://colab.research.google.com/github/hailinwa/Python-projects/blob/main/Sudoku_solver_(GUI).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# type in Conda shell to launch Jupyter Notebook server for Google Colab
jupyter notebook \ --NotebookApp.allow_origin='https://colab.research.google.com' \ --port=8888 \ --NotebookApp.port_retries=0

In [ ]:
#check available system fonts
pygame.font.get_fonts()

In [ ]:
import pygame
import time
pygame.init()
pygame.font.init()

gameFont = 'calibri'
gameFontSize = 40
bgColor = (255,255,128)

# Easy Sudoku
puzzle = [
        [2, 0, 4, 5, 0, 0, 0, 0, 0],
        [0, 5, 3, 9, 0, 7, 0, 0, 0],
        [0, 9, 0, 0, 6, 3, 0, 0, 8],
        [0, 0, 9, 0, 4, 0, 0, 5, 0],
        [0, 8, 0, 7, 9, 5, 0, 1, 0],
        [0, 1, 0, 0, 3, 0, 9, 0, 0],
        [1, 0, 0, 6, 5, 0, 0, 9, 0],
        [0, 0, 0, 8, 0, 2, 1, 3, 0],
        [0, 0, 0, 0, 0, 9, 4, 0, 5]
    ]

# World's hardest Sudoku
# puzzle = [
#         [8, 0, 0, 0, 0, 0, 0, 0, 0],
#         [0, 0, 3, 6, 0, 0, 0, 0, 0],
#         [0, 7, 0, 0, 9, 0, 2, 0, 0],
#         [0, 5, 0, 0, 0, 7, 0, 0, 0],
#         [0, 0, 0, 0, 4, 5, 7, 0, 0],
#         [0, 0, 0, 1, 0, 0, 0, 3, 0],
#         [0, 0, 1, 0, 0, 0, 0, 6, 8],
#         [0, 0, 8, 5, 0, 0, 0, 1, 0],
#         [0, 9, 0, 0, 0, 0, 4, 0, 0]
#     ]

def locBlank(puzzle:'list'):
    # finds a single empty space on the puzzle to be examined by solve()
    for i in range(len(puzzle)): # number of rows
        for j in range(len(puzzle[0])): # number of columns
            if puzzle[i][j] == 0:
                return (i, j)
    return None

def isValid(puzzle:'list', pos:'list', num:'int'):
    # Returns boolean: if the attempted move is valid

    # Check row
    for i in range(len(puzzle)):
        if puzzle[pos[0]][i] == num and pos[1] != i:
            return False

    # Check Col
    for i in range(len(puzzle)):
        if puzzle[i][pos[1]] == num and pos[0] != i:
            return False

    # Check box
    box_x = pos[1]//3
    box_y = pos[0]//3
    for i in range(box_y*3, box_y*3 + 3):
        for j in range(box_x*3, box_x*3 + 3):
            if puzzle[i][j] == num and (i,j) != pos:
                return False
    
    return True

class Cube:
    rows = 9
    cols = 9

    def __init__(self, value, row, col, width, height):
        self.value = value
        self.temp = 0
        self.wrongvalue = 0
        self.row = row
        self.col = col
        self.width = width
        self.height = height
        self.selected = False
        self.redsignal = False

    def draw(self, win):
        fnt = pygame.font.SysFont(gameFont, gameFontSize)

        gap = self.width / 9
        x = self.col * gap
        y = self.row * gap
        
        # Draw typed number into cell
        if self.temp != 0 and self.value == 0:
            text = fnt.render(str(self.temp), 1, (128,128,128))
            win.blit(text, (x+5, y+5))
        elif not(self.value == 0):
            text = fnt.render(str(self.value), 1, (0, 0, 0))
            win.blit(text, (x + (gap/2 - text.get_width()/2), y + (gap/2 - text.get_height()/2)))

        # Draw a red square on selected (clicked) square
        if self.selected:
            pygame.draw.rect(win, (255,0,0), (x, y, gap, gap), 3)

        # Draw red number to indicate incorrectness
        if self.redsignal and self.value == 0:
            text = fnt.render(str(self.wrongvalue), 1, (255, 0, 0))
            win.blit(text, (x + (gap/2 - text.get_width()/2), y + (gap/2 - text.get_height()/2)))

    def draw_change(self, win, g = True):
        fnt = pygame.font.SysFont(gameFont, gameFontSize)

        gap = self.width / 9
        x = self.col * gap
        y = self.row * gap

        pygame.draw.rect(win, bgColor, (x, y, gap, gap), 0)

        text = fnt.render(str(self.value), 1, (0, 0, 0))
        win.blit(text, (x + (gap / 2 - text.get_width() / 2), 
                        y + (gap / 2 - text.get_height() / 2)))
        if g:
            pygame.draw.rect(win, (0, 255, 0), (x, y, gap, gap), 3)
        else:
            pygame.draw.rect(win, (255, 0, 0), (x, y, gap, gap), 3)

    def set(self, val):
        self.value = val
        
    def set_temp(self, val):
        self.temp = val

    def set_wrong(self, val):
        self.wrongvalue = val

class Grid:

    def __init__(self, rows, cols, width, height, win):
        self.rows = rows
        self.cols = cols
        self.cubes = [[Cube(puzzle[i][j], i, j, width, height) 
                    for j in range(cols)] for i in range(rows)]
        self.width = width
        self.height = height
        self.model = None
        self.update_model()
        self.selected = None
        self.win = win

    def update_model(self):
        self.model = [[self.cubes[i][j].value for j in range(self.cols)] 
                      for i in range(self.rows)]

    def sketch(self, val):
        row, col = self.selected
        self.cubes[row][col].set_temp(val)

    def place(self, val):
        # Check with solution if typed number is correct 
        row, col = self.selected
        if self.cubes[row][col].value == 0:
            self.cubes[row][col].set(val)
            self.update_model()

            if isValid(self.model, (row,col), val) and self.solve():
                return True
            else:
                self.cubes[row][col].set(0)
                self.cubes[row][col].set_temp(0)
                self.cubes[row][col].set_wrong(val)
                self.cubes[row][col].redsignal = True
                self.update_model()
                return False

    def draw(self):
        # Draw background grid lines
        gap = self.width / 9
        for i in range(self.rows+1):
            if i % 3 == 0:
                thick = 4
            else:
                thick = 1
            pygame.draw.line(self.win, (0,0,0), (0, i*gap), (self.width, i*gap), thick)
            pygame.draw.line(self.win, (0,0,0), (i*gap, 0), (i*gap, self.height), thick)

        # Draw cubes
        for i in range(self.rows):
            for j in range(self.cols):
                self.cubes[i][j].draw(self.win)

    def select(self, row, col):
        # Reset all cubes to be unselected
        for i in range(self.rows):
            for j in range(self.cols):
                self.cubes[i][j].selected = False
        # Then select the clicked one
        self.cubes[row][col].selected = True
        self.selected = (row, col)

    def clear(self):
        row, col = self.selected
        if self.cubes[row][col].value == 0:
            self.cubes[row][col].set_temp(0)
 
    def click(self, pos):
        # Return the top left corner position of selected (clicked) cell
        if pos[0] < self.width and pos[1] < self.height:
            gap = self.width / 9
            x = pos[0] // gap
            y = pos[1] // gap
            return (int(y),int(x))
        else:
            return None

    def solve(self):
        find = locBlank(self.model)
        if not find:
            return True
        else:
            row, col = find

        for i in range(1, 10):
            if isValid(self.model, (row, col), i):
                self.model[row][col] = i
                if self.solve():
                    return True

                self.model[row][col] = 0

    def solve_gui(self):
        self.update_model()
        find = locBlank(self.model)
        if not find:
            return True
        else:
            row, col = find

        for i in range(1, 10):
            if isValid(self.model, (row, col),i):
                self.model[row][col] = i
                self.cubes[row][col].set(i)
                self.cubes[row][col].draw_change(self.win, True)
                self.update_model()
                pygame.display.update()
                pygame.time.delay(100)

                if self.solve_gui():
                    return True

                self.model[row][col] = 0
                self.cubes[row][col].set(0)
                self.update_model()
                self.cubes[row][col].draw_change(self.win, False)
                pygame.display.update()
                pygame.time.delay(20)

        return False

def redraw_window(win, board):
    win.fill(bgColor)
    # Draw grid and board
    board.draw()

def main():
    win = pygame.display.set_mode((540,540))
    pygame.display.set_caption("Sudoku Solver")
    board = Grid(9, 9, 540, 540, win)
    key = None
    run = True
    start = time.time()
    strikes = 0
    clicked = [4,4]
    while run:

        play_time = round(time.time() - start)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False

            if event.type == pygame.KEYDOWN:
                if event.key in [pygame.K_1, pygame.K_KP1]:
                    key = 1
                if event.key in [pygame.K_2, pygame.K_KP2]:
                    key = 2
                if event.key in [pygame.K_3, pygame.K_KP3]:
                    key = 3
                if event.key in [pygame.K_4, pygame.K_KP4]:
                    key = 4
                if event.key in [pygame.K_5, pygame.K_KP5]:
                    key = 5
                if event.key in [pygame.K_6, pygame.K_KP6]:
                    key = 6
                if event.key in [pygame.K_7, pygame.K_KP7]:
                    key = 7
                if event.key in [pygame.K_8, pygame.K_KP8]:
                    key = 8
                if event.key in [pygame.K_9, pygame.K_KP9]:
                    key = 9

                if event.key == pygame.K_DELETE:
                    board.clear()
                    key = None

                if event.key == pygame.K_LEFT:
                    clicked = (clicked[0], max(0,clicked[1]-1))
                    board.select(clicked[0], clicked[1])
                    key = None
                if event.key == pygame.K_RIGHT:
                    clicked = (clicked[0], min(8,clicked[1]+1))
                    board.select(clicked[0], clicked[1])
                    key = None
                if event.key == pygame.K_UP:
                    clicked = (max(0,clicked[0]-1), clicked[1])
                    board.select(clicked[0], clicked[1])
                    key = None
                if event.key == pygame.K_DOWN:
                    clicked = (min(8,clicked[0]+1), clicked[1])
                    board.select(clicked[0], clicked[1])
                    key = None

                if event.key == pygame.K_SPACE:
                    board.solve_gui()

                # Check if typed number is correct
                if event.key == pygame.K_RETURN:
                    i, j = board.selected
                    if board.cubes[i][j].temp != 0:
                        board.place(board.cubes[i][j].temp)
                        key = None

            # Record position of mouse click and pass to (row,col)
            if event.type == pygame.MOUSEBUTTONDOWN:
                pos = pygame.mouse.get_pos()
                clicked = board.click(pos)
                if clicked:
                    board.select(clicked[0], clicked[1])
                    key = None

        # Store typed value to "temp" attribute of cubes object
        if board.selected and key != None:
            board.sketch(key)

        redraw_window(win, board)
        pygame.display.update()

main()
pygame.quit()